In [ ]:
from mineSweeper import MinesweeperEnv
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
import numpy as np

def linear_schedule(initial_value, final_value=0.0):

    if isinstance(initial_value, str):
        initial_value = float(initial_value)
        final_value = float(final_value)
        assert (initial_value > 0.0)

    def scheduler(progress):
        return final_value + progress * (initial_value - final_value)

    return scheduler

NUM_ENVS = 32
make_env = lambda seed=None: Monitor(MinesweeperEnv(9, 9, 9, seed=seed), info_keywords=("is_success",))
# env = SubprocVecEnv([make_env(seed=s) for s in np.random.randint(1,1e9, NUM_ENVS)])
env = DummyVecEnv([make_env])
# env = ActionMasker(env, MinesweeperEnv.get_action_mask)
lr_schedule = linear_schedule(2.5e-4, 2.5e-6)
clip_range_schedule = linear_schedule(0.15, 0.025)
if True:
    model = MaskablePPO(
        "MlpPolicy", 
        env=env, 
        batch_size=2048,
        policy_kwargs={"net_arch" : [256, 512, 1024, 2048, 2048, 1024, 512, 256]},
        verbose=1,
        tensorboard_log="./tensorboard/",
        learning_rate=lr_schedule,
        clip_range=clip_range_schedule,
        device='cuda',
    )
else:
    model = MaskablePPO.load("./model/mlp.pkl", env=env)
model.learn(total_timesteps=1e7)


In [ ]:
model.save("./model/mlp.pkl")

In [ ]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
env.close()
mean_reward, std_reward